<a href="https://colab.research.google.com/github/djps/k-wave-python/blob/pstdelastic3D/examples/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/waltsims/k-wave-python

import numpy as np
import cupy as cp

from kwave.data import Vector
from kwave.kgrid import kWaveGrid

def accelerated_fft_operation(ddx_k_shift_pos, x):
    xp = cp.get_array_module(x)
    print(xp)
    x_fft = xp.fft.fft(x, axis=0)
    result_fft = xp.multiply(ddx_k_shift_pos, x_fft)
    result_ifft = xp.fft.ifft(result_fft, axis=0)
    result_real =  xp.real(result_ifft)
    return result_real

  Cloning https://github.com/waltsims/k-wave-python to /tmp/pip-req-build-1zkvugs2
  Running command git clone --filter=blob:none --quiet https://github.com/waltsims/k-wave-python /tmp/pip-req-build-1zkvugs2
  Resolved https://github.com/waltsims/k-wave-python to commit bf66fad4e2385dd254d0df805cacffea7e0c952d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
Nx = 100
Ny = 100

grid_size_points = Vector([Nx, Ny])

dx = 0.01
dy = 0.01

# vector of resolution of grid
grid_spacing_meters = Vector([dx, dy])

# create the k-space grid
kgrid = kWaveGrid(grid_size_points, grid_spacing_meters)

kx_vec, ky_vec = kgrid.k_vec
kx_vec, ky_vec = np.array(kx_vec), np.array(ky_vec)

ddx_k_shift_pos =  np.fft.ifftshift( 1j * kx_vec )

temp = np.ones_like(kx_vec)

# Assuming ddx_k_shift_pos and temp are numpy arrays, convert them to cupy arrays
ddx_k_shift_pos_gpu = cp.asarray(ddx_k_shift_pos)
temp_gpu = cp.asarray(temp)

# Perform FFT, element-wise multiplication, and inverse FFT using CuPy on the GPU
temp_fft_gpu = cp.fft.fft(temp_gpu, axis=0)
result_fft_gpu = cp.multiply(ddx_k_shift_pos_gpu, temp_fft_gpu)
result_ifft_gpu = cp.fft.ifft(result_fft_gpu, axis=0)
result_real_gpu = cp.real(result_ifft_gpu)

# Convert the result back to a NumPy array if necessary
result_real = cp.asnumpy(result_real_gpu)

result = accelerated_fft_operation(ddx_k_shift_pos, temp)

print(result - result_real)

<module 'numpy' from '/usr/local/lib/python3.10/dist-packages/numpy/__init__.py'>
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
